In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
pip install -q transformers

In [ ]:
pip install -q lime

In [ ]:
import pandas as pd
import torch
import pickle
from lime.lime_text import LimeTextExplainer
from torch.nn import functional as F

In [ ]:
device = torch.device("cuda")

In [ ]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [ ]:
test =  pd.read_csv('/content/drive/MyDrive/MasterThesis/Dataset/X_test_all.csv')
test_sentences = test['clean_review']

In [ ]:
model = torch.load('/content/drive/MyDrive/MasterThesis/BERT/bert_1', map_location="cuda:0")

In [ ]:
explainer = LimeTextExplainer(class_names=["Negative", "Positive"])

In [ ]:
#texts = test['clean_review'].to_list()

def new_predict(texts):
    tokens = tokenizer(texts, return_tensors='pt', padding=True)
    output = model(tokens['input_ids'].to(device), tokens['attention_mask'].to(device))[0]
    tensor_logits = output
    probas = F.softmax(tensor_logits, dim=-1).cpu().detach().numpy()
    return probas

In [ ]:
#i = 3145
#exp = explainer.explain_instance(test_sentences.iloc[i], new_predict, num_features=30, num_samples=100)  #number of samples is small
#exp.show_in_notebook()

In [ ]:
#s = 'place consistent pizzas came looking exactly like yelp pictures afraid single piece pizza entire center pizza crust completely soggy second piece pizza cold not mention toppings skimpy haphazard not close representing art pieces lamb sausage pizza sausage mushroom leek pizza got super bland liberal dose hot pepper flakes ditch failed effort majority remained limp cold pies crust recipe testing'

In [ ]:
exp = explainer.explain_instance(test_sentences.iloc[2047], new_predict, num_features=50, num_samples=100) 
exp_list = exp.as_list()
sorted_list = sorted(exp_list, key=lambda x: x[1])
#lime_calc.append(sorted_list)
sorted_list

[('stay', -0.0342065081360304),
 ('experience', -0.02847821743496565),
 ('free', -0.025224831669638032),
 ('couldn', -0.02504014438487464),
 ('god', -0.021007830695492186),
 ('away', -0.019550693055399335),
 ('terrible', -0.018417781225359288),
 ('lie', -0.016586282432971795),
 ('worst', -0.014131741203752834),
 ('possible', -0.013334565657789357),
 ('horrible', -0.012302214894240387),
 ('smelled', -0.012136710240631428),
 ('disgusting', -0.011600693148084958),
 ('situation', -0.004218969522827772),
 ('cockroaches', -0.003206733880870698),
 ('place', -0.0007976608686268581),
 ('not', 0.0056844426411481195),
 ('know', 0.006850957445118752),
 ('absolutely', 0.008790734698605094),
 ('big', 0.009056970433711386),
 ('room', 0.009544966990922102),
 ('omg', 0.010415628439013275),
 ('mention', 0.013809795027624435),
 ('stayed', 0.015836111155572968),
 ('thank', 0.016301572523908917),
 ('wifi', 0.01865746617995945),
 ('night', 0.019496914848420126),
 ('hours', 0.021990444516336433),
 ('sleep', 

**Sample calculation**

Calculating and saving feature importance score for the reviews used as a demonstration example to measure the proposed criteria.

In [ ]:
pos_sample = pickle.load(open('/content/drive/MyDrive/MasterThesis/Dataset/positive_sample_50', 'rb'))
neg_sample = pickle.load(open('/content/drive/MyDrive/MasterThesis/Dataset/negative_sample_50', 'rb')) 

In [ ]:
def lime_calc(sample):
  lime_calc = []
  for i in sample:
    exp = explainer.explain_instance(test_sentences.iloc[i], new_predict, num_features=50, num_samples=100) 
    exp_list = exp.as_list()
    sorted_list = sorted(exp_list, key=lambda x: x[1])
    lime_calc.append(sorted_list)
  return lime_calc

In [ ]:
r = lime_calc(pos_sample) 
with open('/content/drive/MyDrive/MasterThesis/Dataset/lime_bert_positive_sample_50', 'wb') as f:
    pickle.dump(r, f) 

r = lime_calc(neg_sample)
with open('/content/drive/MyDrive/MasterThesis/Dataset/lime_bert_negative_sample_50', 'wb') as f:
    pickle.dump(r, f)        